In [1]:
import os

In [2]:
%pwd

'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'd:\\Code\\Git\\myprojects\\Equation-Solver-End-to-End\\Handwritten_Equation_Solver'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from equation_solver.constants import *
from equation_solver.utils.common import read_yaml, create_directories
import tensorflow as tf

[2024-11-06 13:07:28,817: WARNING: module_wrapper: From c:\Users\anand\anaconda3\envs\myenv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "symbol_data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.batch_size = self.config.params_batch_size
        print(f"Using batch size: {self.batch_size}")

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        input_shape = self.model.input_shape
        print(f"Model expects input shape: {input_shape}")
        self.model.summary()

    def train_valid_generator(self):
        input_shape = self.model.input_shape
        color_mode = "grayscale" if input_shape[-1] == 1 else "rgb"
        # print(f"Using color mode: {color_mode}")

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.batch_size,
            interpolation="bilinear",
            color_mode=color_mode,
            class_mode='sparse'  # For sparse categorical crossentropy
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Validation generator - no shuffling
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        # Training generator - with shuffling
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # Print generator details for debugging
        print(f"Training data shape: {self.train_generator.image_shape}")
        print(f"Number of classes: {len(self.train_generator.class_indices)}")
        print(f"Training batch size: {self.train_generator.batch_size}")
        print(f"Steps per epoch: {len(self.train_generator)}")

    def train(self):
        # Calculate steps properly
        self.steps_per_epoch = len(self.train_generator)
        self.validation_steps = len(self.valid_generator)

        # Compile the model with appropriate loss and metrics
        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        # Train the model
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            verbose=1
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
        return history

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2024-11-06 13:07:30,126: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 13:07:30,129: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-06 13:07:30,132: INFO: common: created directory at: artifacts]
[2024-11-06 13:07:30,133: INFO: common: created directory at: artifacts\training]
Using batch size: 128
[2024-11-06 13:07:30,470: WARNING: module_wrapper: From c:\Users\anand\anaconda3\envs\myenv\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
Model expects input shape: (None, 45, 45, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 45, 45, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 45, 45, 32) 

c:\Users\anand\anaconda3\envs\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
